# Selecting and Splitting Data

In [1]:
from bs4 import BeautifulSoup
import os
import numpy as np
import json
import pickle
from sklearn.model_selection import train_test_split

In [8]:
def get_bnbox(soup):
    objs = soup.find_all('object')
    golds = {}
    for obj in objs:
        names = obj.find_all('name')
        bndboxes = obj.find_all('bndbox')
        if len(bndboxes) > 0:
            boxes = [get_coordinates(box) for box in bndboxes]
            for name in names:
                if name.string in golds:
                    golds[name.string].extend(boxes)
                else:
                    golds[name.string] = boxes
    return golds


def get_coordinates(bnbox):
    # takes the xml coordinates and returns a tuple (ymin, xmin, ymax, xmax)
    ymin = int(bnbox.find('ymin').string)
    xmin = int(bnbox.find('xmin').string)
    ymax = int(bnbox.find('ymax').string)
    xmax = int(bnbox.find('xmax').string)
    # debug
    #print(ymin, xmin, ymax, xmax)
    #print(type(ymin), type(xmin), type(ymax), type(xmax))
    return (ymin, xmin, ymax, xmax)


## helper function to turn a numpy unsigned box into a python tuple of (normal,
# signed) python integers
def as_int_tuple(np_box):
    box = [int(i) for i in np_box]
    return tuple(box)


## Function to remove objects from the dictionary that appear multiple times
#  in an image
def remove_multiple_objs(dict_obj):
    single_objs_by_file = {}
    
    for file in dict_obj.keys():
        single_objs_by_file[file] = {}
        
        for obj in dict_obj[file].keys():
            if len(dict_obj[file][obj]) == 1:
                single_objs_by_file[file][obj] = dict_obj[file][obj]
        
    return single_objs_by_file

def get_obj_names(text):
    temp = {}
    d = {}
    
    closing_bracket = 0
    while text.find("#", closing_bracket) != -1:
        hash_sign = text.find("#", closing_bracket) + 1
        closing_bracket = text.find("]", hash_sign)
        
        description = text[hash_sign:closing_bracket].lower().split("/")
        obj = description[0]
        phrase = description[-1]
        phrase = phrase.split(" ")[1:]
        
        if obj in temp:
            temp[obj].extend(phrase)
        else:            
            temp[obj] = phrase
    
    stopwords = ["a","an","the","his","her","their","one","two","three","four","five","six","seven","eight",\
                "nine","ten","blue","red","yellow","black","white","orange","green","purple","pink","gold","silver"]
    
    for obj in temp.keys():
        l = []
        c = []
        for word in temp[obj]:
            if word in stopwords:
                continue
            elif word not in l:
                l.append(word)
                c.append(1)
            else:
                c[l.index(word)] += 1
        
        if len(l) > 0:
            if c.count(max(c)) == 1:
                d[obj] = {"word":l[np.argmax(c)]}
    return d


def remove_categories(dict_obj, obj_cat):
    
    new_dict_obj = {}
    
    for file in dict_obj.keys():
        new_dict_obj[file] = {}
        used_cat = []
        
        for obj in dict_obj[file].keys():
        # Categories that occur more than once are removed:
            if obj not in obj_cat[file]:
                print("Obj not in obj_cat file: {}, obj: {}".format(file,obj))
                continue
            elif obj_cat[file][obj] in used_cat:
                continue
        # Categories "other" and "scene" are removed:
            elif obj_cat[file][obj] == "other":
                continue
            elif obj_cat[file][obj] == "scene":
                continue
            else:
                used_cat.append(obj_cat[file][obj])
                l = []
                l.append(dict_obj[file][obj])
                l.append(obj_cat[file][obj])
                new_dict_obj[file][obj] = l
        
        # Images with only 1 remaining object are removed:
        if len(new_dict_obj[file]) < 2:
            del new_dict_obj[file]

    return new_dict_obj

## Sentences

### Extract object names from `Sentences`.

In [3]:
sents = os.listdir("F30k/Flickr30kEntities/Sentences")
obj_names_by_file = {}

for sent in sents:
    file_name = sent.split(".")[0]
    
    with open(("F30k/Flickr30kEntities/Sentences/"+file_name+".txt"), "r") as f:
        contents = f.read()
        
        obj_names_by_file[file_name] = get_obj_names(contents)

obj_names_by_file 

{'3187924573': {'0': {'word': 'it'},
  '99600': {'word': 'bird'},
  '99601': {'word': 'seagulls'},
  '99602': {'word': 'beak'},
  '99604': {'word': 'water'},
  '99605': {'word': 'fish'}},
 '2856053254': {'77019': {'word': 'boy'},
  '77022': {'word': 'park'},
  '77024': {'word': 'hands'},
  '77025': {'word': 'ladder'}},
 '445242295': {'169815': {'word': 'boat'},
  '169817': {'word': 'water'},
  '169818': {'word': 'shore'},
  '169819': {'word': 'man'}},
 '298883053': {'0': {'word': 'he'},
  '85802': {'word': 'plastic'},
  '85803': {'word': 'legos'},
  '85805': {'word': 'playpen'}},
 '284644694': {'76591': {'word': 'girl'},
  '76593': {'word': 'man'},
  '76594': {'word': 'spectators'},
  '76597': {'word': 'toothbrush'},
  '76598': {'word': 'paste'},
  '76599': {'word': 'india'}},
 '2464118785': {'0': {'word': 'that'},
  '51429': {'word': 'women'},
  '51434': {'word': 'woman'},
  '51436': {'word': 'target'},
  '51437': {'word': 'glass'},
  '51439': {'word': 'game'},
  '51440': {'word': 'ca

## Annotations

### Create a Dictionary of Objects

In [14]:
annots = os.listdir("F30k/Flickr30kEntities/Annotations")

dict_of_objects = {} # {filename: {objects: [coordinates]}

for ann in annots:
    file_name = ann.split(".")[0]
    with open(("F30k/Flickr30kEntities/Annotations/"+file_name+".xml"), "r") as f:
        contents = f.read()

    soup = BeautifulSoup(contents, "xml")
    
    dict_of_objects[file_name] = get_bnbox(soup)

## Remove Multiple Objects

In [18]:
# Applying remove_multiple_objs:
dict_of_objects = remove_multiple_objs(dict_of_objects)

31783


### Putting the Dictionaries Together

In [53]:
dict_words_boxes = {}
# ALSO REMOVEs MULTIPLE IDENTICAL WORDS FROM THE SAME IMAGE

for file in obj_names_by_file.keys():
    dict_words_boxes[file] = {}
    used_words = []
    
    for obj in obj_names_by_file[file].keys():
        
        word = obj_names_by_file[file][obj]["word"]
        if word in used_words:
            continue # SKIPS OBJECTS WITH A USED WORD. NOT RANDOMLY SELECTED!
        else:
            used_words.append(word)
        
        if obj in dict_of_objects[file]:
            dict_words_boxes[file][obj] = {}         
            dict_words_boxes[file][obj]["word"] = obj_names_by_file[file][obj]["word"]
            dict_words_boxes[file][obj]["bnbox"] = dict_of_objects[file][obj]
    
    if len(dict_words_boxes[file]) < 2: # REMOVES IMAGES WITH FEWER THAN 2 OBJECTS REMAINING
        del dict_words_boxes[file]

File: 2464118785
{'51436': {'word': 'target', 'bnbox': [(29, 370, 151, 499)]}}

File: 7006590104
{'263824': {'word': 'ball', 'bnbox': [(259, 34, 301, 77)]}}

File: 16397322
{'18001': {'word': 'woman', 'bnbox': [(157, 14, 272, 93)]}}

File: 3599124739
{'130549': {'word': 'surf', 'bnbox': [(76, 1, 375, 499)]}}

File: 110600869
{'2672': {'word': 'jacket', 'bnbox': [(136, 4, 338, 319)]}}

File: 2468906010
{}

File: 3271495320
{'105961': {'word': 'waterside', 'bnbox': [(126, 1, 343, 407)]}}

File: 717673249
{'266598': {'word': 'grass', 'bnbox': [(1, 1, 332, 498)]}}

File: 4099476662
{'155392': {'word': 'man', 'bnbox': [(10, 132, 473, 473)]}}

File: 4104001356
{}

File: 2040964486
{}

File: 4891714957
{'215597': {'word': 'man', 'bnbox': [(19, 3, 353, 289)]}}

File: 3956420030
{'149504': {'word': 'bench', 'bnbox': [(357, 1, 472, 334)]}}

File: 4242041141
{'160116': {'word': 'softball', 'bnbox': [(327, 432, 363, 468)]}}

File: 799431781
{'277480': {'word': 'mouth', 'bnbox': [(130, 210, 154, 26

File: 3344692671
{'111315': {'word': 'vests', 'bnbox': [(143, 226, 241, 300)]}}

File: 188552459
{'22614': {'word': 'poster', 'bnbox': [(23, 111, 358, 343)]}}

File: 2764057744
{'71350': {'word': 'pit', 'bnbox': [(247, 89, 373, 276)]}}

File: 878758390
{}

File: 4940289938
{'221041': {'word': 'woman', 'bnbox': [(86, 263, 400, 500)]}}

File: 1467533293
{'14020': {'word': 'pink', 'bnbox': [(140, 168, 309, 225)]}}

File: 377872472
{'142296': {'word': 'dog', 'bnbox': [(105, 1, 169, 73)]}}

File: 3105691757
{}

File: 3415589320
{'116472': {'word': 'grass', 'bnbox': [(2, 3, 374, 500)]}}

File: 4015350856
{}

File: 4361532957
{}

File: 501699433
{'227064': {'word': 'child', 'bnbox': [(129, 332, 331, 499)]}}

File: 3184031654
{'99287': {'word': 'woman', 'bnbox': [(77, 242, 320, 480)]}}

File: 145291221
{'13413': {'word': 'sign', 'bnbox': [(127, 216, 175, 280)]}}

File: 4075205914
{'154337': {'word': 'others', 'bnbox': [(106, 56, 331, 499)]}}

File: 1457762320
{'13604': {'word': 'water', 'bnbox

File: 1461329041
{'13753': {'word': 'concrete', 'bnbox': [(225, 1, 371, 500)]}}

File: 3169276423
{'97942': {'word': 'woman', 'bnbox': [(52, 228, 332, 451)]}}

File: 2301379282
{'40059': {'word': 'baby', 'bnbox': [(49, 103, 392, 318)]}}

File: 4726714032
{}

File: 239807547
{}

File: 7425128736
{}

File: 908636680
{'283363': {'word': 'marshmallows', 'bnbox': [(240, 146, 253, 159)]}}

File: 3343197133
{'111151': {'word': 'skyline', 'bnbox': [(4, 2, 273, 499)]}}

File: 835606668
{'281179': {'word': 'people', 'bnbox': [(116, 2, 246, 134)]}}

File: 2774504145
{}

File: 4689716914
{}

File: 2772084628
{}

File: 4612024952
{}

File: 4629445758
{'184345': {'word': 'table', 'bnbox': [(241, 282, 334, 500)]}}

File: 97105139
{}

File: 6302275137
{'252036': {'word': 'competitor', 'bnbox': [(22, 199, 338, 487)]}}

File: 3468694409
{'120344': {'word': 'station', 'bnbox': [(7, 13, 194, 494)]}}

File: 76466808
{'272649': {'word': 'plane', 'bnbox': [(74, 20, 300, 499)]}}

File: 4671423167
{}

File: 46

{'185042': {'word': 'umbrella', 'bnbox': [(1, 238, 169, 415)]}}

File: 2705099283
{}

File: 4503659154
{'173586': {'word': 'couple', 'bnbox': [(182, 39, 500, 323)]}}

File: 4889806295
{'215254': {'word': 'man', 'bnbox': [(108, 225, 333, 349)]}}

File: 4752961136
{'198824': {'word': 'man', 'bnbox': [(44, 99, 309, 233)]}}

File: 2656914235
{}

File: 1084040636
{}

File: 4792134256
{'203181': {'word': 'table', 'bnbox': [(246, 1, 375, 499)]}}

File: 6948564341
{'262247': {'word': 'net', 'bnbox': [(43, 166, 264, 345)]}}

File: 3776839227
{}

File: 4716793596
{'194401': {'word': 'statue', 'bnbox': [(15, 220, 237, 345)]}}

File: 4571895229
{'179740': {'word': 'area', 'bnbox': [(1, 1, 232, 500)]}}

File: 2971381841
{'84777': {'word': 'island', 'bnbox': [(240, 46, 334, 383)]}}

File: 3431487300
{}

File: 22930048
{'39584': {'word': 'red', 'bnbox': [(67, 109, 230, 203)]}}

File: 2424556844
{'48444': {'word': 'blond', 'bnbox': [(127, 179, 375, 299)]}}

File: 2937365149
{}

File: 189740668
{'22868

File: 369047365
{}

File: 4701498512
{'192822': {'word': 'hats', 'bnbox': [(75, 293, 208, 481)]}}

File: 2782759274
{'72748': {'word': 'room', 'bnbox': [(1, 1, 333, 500)]}}

File: 2493825916
{'53203': {'word': 'boy', 'bnbox': [(66, 171, 271, 276)]}}

File: 2864854530
{'77613': {'word': 'women', 'bnbox': [(191, 311, 316, 387)]}}

File: 3168841415
{'97867': {'word': 'frisbee', 'bnbox': [(175, 206, 200, 265)]}}

File: 4862788297
{'212508': {'word': 'girl', 'bnbox': [(6, 5, 332, 294)]}}

File: 2087954558
{'27851': {'word': 'leader', 'bnbox': [(248, 143, 333, 180)]}}

File: 3473320907
{'120760': {'word': 'face', 'bnbox': [(287, 84, 304, 96)]}}

File: 3446191973
{}

File: 4545817922
{}

File: 3780768589
{'142358': {'word': 'building', 'bnbox': [(1, 2, 333, 500)]}}

File: 4888378070
{}

File: 277167533
{}

File: 3375070563
{'113943': {'word': 'white', 'bnbox': [(28, 122, 229, 334)]}}

File: 2980958891
{'85335': {'word': 'building', 'bnbox': [(4, 4, 404, 363)]}}

File: 242324909
{'48324': {'wo

...


File: 7185642084
{'266750': {'word': 'banner', 'bnbox': [(62, 43, 183, 400)]}}

File: 3643684044
{'133729': {'word': 'table', 'bnbox': [(237, 35, 331, 494)]}}

File: 2542415282
{}

File: 4805567702
{'204711': {'word': 'rv', 'bnbox': [(2, 5, 247, 323)]}}

File: 3231749680
{'102907': {'word': 'floor', 'bnbox': [(87, 1, 374, 500)]}}

File: 2354064281
{}

File: 4491304968
{'172842': {'word': 'hands', 'bnbox': [(261, 214, 295, 248)]}}

File: 241347114
{}

File: 4445485475
{'168992': {'word': 'building', 'bnbox': [(44, 34, 241, 455)]}}

File: 134653574
{'9351': {'word': 'women', 'bnbox': [(191, 19, 493, 235)]}}

File: 2209317
{'34270': {'word': 'black', 'bnbox': [(217, 388, 333, 474)]}}

File: 3551717658
{}

File: 3609061538
{'131304': {'word': 'road', 'bnbox': [(179, 1, 284, 164)]}}

File: 4718146904
{}

File: 5536018957
{'238438': {'word': 'roof', 'bnbox': [(115, 3, 333, 500)]}}

File: 3855516061
{'145787': {'word': 'white', 'bnbox': [(87, 324, 173, 433)]}}

File: 363701716
{'133190': {'w

File: 5846714531
{'243988': {'word': 'person', 'bnbox': [(214, 431, 259, 458)]}}

File: 152809468
{'15686': {'word': 'bird', 'bnbox': [(117, 224, 226, 331)]}}

File: 450138714
{'173463': {'word': 'table', 'bnbox': [(238, 80, 375, 500)]}}

File: 229059021
{'39407': {'word': 'bench', 'bnbox': [(89, 88, 270, 434)]}}

File: 4603713542
{'181837': {'word': 'scarf', 'bnbox': [(38, 247, 273, 347)]}}

File: 3670681503
{'135701': {'word': 'mother', 'bnbox': [(109, 418, 331, 499)]}}

File: 4773293976
{'201379': {'word': 'black', 'bnbox': [(27, 334, 142, 449)]}}

File: 4935686055
{'220432': {'word': 'woman', 'bnbox': [(53, 442, 298, 500)]}}

File: 3719461451
{}

File: 4718134968
{'194822': {'word': 'street', 'bnbox': [(241, 2, 499, 498)]}}

File: 2537000363
{}

File: 2536991887
{}

File: 2831394884
{'75396': {'word': 'man', 'bnbox': [(1, 1, 324, 398)]}}

File: 2091171488
{'28174': {'word': 'woman', 'bnbox': [(17, 11, 499, 411)]}}

File: 4683565867
{'190116': {'word': 'blue', 'bnbox': [(235, 1, 332

File: 3722006169
{'139567': {'word': 'car', 'bnbox': [(107, 96, 251, 375)]}}

File: 4631909374
{'184673': {'word': 'man', 'bnbox': [(271, 77, 387, 156)]}}

File: 2340111592
{'42704': {'word': 'grass', 'bnbox': [(56, 5, 374, 498)]}}

File: 535309053
{'234382': {'word': 'woman', 'bnbox': [(186, 179, 248, 209)]}}

File: 4930823945
{'219614': {'word': 'other', 'bnbox': [(54, 310, 330, 427)]}}

File: 2260369648
{'37440': {'word': 'woman', 'bnbox': [(2, 299, 440, 500)]}}

File: 4711637472
{}

File: 3173461705
{'98244': {'word': 'fence', 'bnbox': [(3, 1, 240, 500)]}}

File: 2596900481
{'60444': {'word': 'basket', 'bnbox': [(195, 51, 290, 121)]}}

File: 4793108729
{}

File: 3385593926
{'114602': {'word': 'fence', 'bnbox': [(71, 1, 158, 499)]}}

File: 3534592364
{'125633': {'word': 'children', 'bnbox': [(122, 2, 375, 500)]}}

File: 141364977
{'11916': {'word': 'blond', 'bnbox': [(60, 113, 138, 179)]}}

File: 4437028723
{}

File: 32628083
{}

File: 2285664
{'39003': {'word': 'sidewalk', 'bnbox':

File: 43244430
{}

File: 3357209547
{}

File: 6295775965
{}

File: 4891975016
{'215617': {'word': 'street', 'bnbox': [(116, 3, 423, 379)]}}

File: 3374776293
{'113924': {'word': 'grass', 'bnbox': [(222, 1, 375, 500)]}}

File: 286797291
{'77815': {'word': 'sidewalk', 'bnbox': [(211, 1, 375, 500)]}}

File: 7396941510
{'269342': {'word': 'orange', 'bnbox': [(219, 102, 338, 281)]}}

File: 396154083
{'149778': {'word': 'woman', 'bnbox': [(51, 352, 115, 462)]}}

File: 2468921460
{'51721': {'word': 'net', 'bnbox': [(80, 237, 199, 497)]}}

File: 4810830329
{}

File: 6993511259
{'263453': {'word': 'man', 'bnbox': [(36, 176, 415, 276)]}}

File: 5381043783
{'235000': {'word': 'street', 'bnbox': [(258, 82, 374, 500)]}}

File: 3300679815
{}

File: 7669392800
{'273022': {'word': 'road', 'bnbox': [(287, 1, 500, 500)]}}

File: 6829605875
{}

File: 98377566
{'284907': {'word': 'cage', 'bnbox': [(1, 27, 500, 459)]}}

File: 2536978777
{}

File: 5973146939
{'246295': {'word': 'wicketkeeper', 'bnbox': [(13

File: 5458061796
{}

File: 968081289
{'284487': {'word': 'water', 'bnbox': [(89, 1, 499, 433)]}}

File: 2291958125
{'39518': {'word': 'calf', 'bnbox': [(220, 169, 435, 346)]}}

File: 3065560742
{'91101': {'word': 'ice', 'bnbox': [(210, 3, 385, 500)]}}

File: 6872034011
{'259990': {'word': 'snow', 'bnbox': [(104, 1, 333, 500)]}}

File: 4545487394
{'176926': {'word': 'bicycle', 'bnbox': [(181, 190, 374, 322)]}}

File: 2685518281
{}

File: 3677927146
{'136268': {'word': 'grass', 'bnbox': [(100, 1, 333, 500)]}}

File: 2427094466
{}

File: 4902710339
{}

File: 3159569570
{}

File: 3251646144
{'104483': {'word': 'large', 'bnbox': [(1, 1, 422, 500)]}}

File: 4746185365
{}

File: 2766926202
{'71564': {'word': 'water', 'bnbox': [(165, 5, 333, 499)]}}

File: 6184114561
{'249422': {'word': 'field', 'bnbox': [(157, 4, 373, 499)]}}

File: 2916012955
{'81167': {'word': 'gate', 'bnbox': [(87, 292, 159, 392)]}}

File: 3498240367
{'122813': {'word': 'picture', 'bnbox': [(30, 85, 500, 333)]}}

File: 437

## Saving the Dictionary with Words and Boxes

The bounding boxes were included for an earlier form of the model. In the eventual model, the bounding boxes were no longer necessary.

In [ ]:
json_file = json.dumps(dict_words_boxes)
f = open("dict_words_boxes.json","w")
f.write(json_file)
f.close()

## Splitting Data

In [59]:
files = list(cleaned_dict.keys())

train_data, test_data = train_test_split(files, test_size= 1000)
train_data, validation_data = train_test_split(train_data, test_size= 1000)

'3187924573'

In [62]:
print(len(train_data))
print(train_data[:5])

21545
['3352791995', '2905250733', '870186901', '5188555891', '3201845131']


In [63]:
print(len(validation_data))
print(validation_data[:5])

1000
['4461621138', '4721533619', '4053707091', '7666764962', '4756096577']


In [64]:
print(len(test_data))
print(test_data[:5])

1000
['3483140026', '4467946265', '6156276992', '338857661', '175556963']


### Saving Splits

In [65]:
with open("train_data.txt", "wb") as fp:
    pickle.dump(train_data, fp)

with open("validation_data.txt", "wb") as fp:
    pickle.dump(validation_data, fp)

with open("test_data.txt", "wb") as fp:
    pickle.dump(test_data, fp)

In [68]:
print(train_data2[300])
cleaned_dict[train_data2[300]]

4728183104


{'196117': {'bnbox': [[20, 154, 374, 277]], 'word': 'man'},
 '196118': {'bnbox': [[110, 143, 146, 169]], 'word': 'beer'},
 '196119': {'bnbox': [[160, 101, 192, 121]], 'word': 'glass'},
 '196122': {'bnbox': [[150, 77, 194, 161]], 'word': 'food'},
 '196123': {'bnbox': [[97, 135, 166, 180]], 'word': 'tap'},
 '196124': {'bnbox': [[167, 83, 405, 209]], 'word': 'cart'}}